In [2]:
#S&P 400
import ssl
import time

ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
def getsp400():
    
    sp400_tickers=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')
    sp400_tickers= sp400_tickers[0]

    tickers = sp400_tickers["Ticker symbol"].values.tolist()
    return tickers
companies = getsp400()
companies.insert(0,'SP400')
#df=table[0]
#df.to_csv('S&P400-Info.csv')
#df.to_csv('S&P400-Symbols.csv', columns=['Ticker symbol'])


In [3]:
companies

['SP400',
 'ACC',
 'ACHC',
 'ACIW',
 'ACM',
 'ADS',
 'ADNT',
 'AEO',
 'AFG',
 'AGCO',
 'AIRC',
 'ALE',
 'AM',
 'AMED',
 'AMG',
 'AMKR',
 'AN',
 'APPS',
 'ARW',
 'ARWR',
 'ASB',
 'ASGN',
 'ASH',
 'ATR',
 'AVNT',
 'AVT',
 'AYI',
 'AXON',
 'AZPN',
 'BC',
 'BCO',
 'BDC',
 'BJ',
 'BKH',
 'BLD',
 'BLDR',
 'BLKB',
 'BOH',
 'BRKS',
 'BYD',
 'BHF',
 'BRX',
 'CABO',
 'CACI',
 'CADE',
 'CAR',
 'CCMP',
 'CASY',
 'CATY',
 'CBRL',
 'CBSH',
 'CBT',
 'CC',
 'CDK',
 'CFR',
 'CFX',
 'CGNX',
 'CHE',
 'CHH',
 'CHDN',
 'CIEN',
 'CHX',
 'CIT',
 'CLF',
 'CLH',
 'CMC',
 'CMP',
 'CNO',
 'CNX',
 'CNXC',
 'COHR',
 'COTY',
 'CONE',
 'COLM',
 'COR',
 'CPRI',
 'CPT',
 'CR',
 'CRI',
 'CRNC',
 'CROX',
 'CRUS',
 'CSL',
 'CUZ',
 'CVLT',
 'CW',
 'DAN',
 'DAR',
 'DCI',
 'DECK',
 'DEI',
 'DKS',
 'DTM',
 'DY',
 'EGP',
 'EHC',
 'ELY',
 'EME',
 'EBS',
 'ENR',
 'ENS',
 'ENV',
 'EPR',
 'EQT',
 'ETRN',
 'ESNT',
 'EVR',
 'EWBC',
 'EXEL',
 'EXP',
 'FAF',
 'FDS',
 'FCFS',
 'FFIN',
 'FHN',
 'FICO',
 'FHI',
 'FIVE',
 'FLO',
 'FLR',


In [17]:
import requests
import pandas as pd

price = {}
metrics = {}
count = 0
for company in companies:
    count += 1
    time.sleep(13)
    print('count ', count)
    try:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={company}&outputsize=full&apikey=LPUJVURUAHSRJQ5J'
        r = requests.get(url)
        prices_retrieval = r.json()
        
        price[company] = {}
        metrics[company] = {} 
        
        for item in prices_retrieval:
            #to check that the API call is valid
            if item != 'Error Message':
                retrieval = prices_retrieval['Time Series (Daily)']
                #sets the date and price for each ticket
                for t in retrieval:
                    price_date = t
                    price[company][price_date] = retrieval[t]['4. close']
            else:
                pass
        price_DF = pd.DataFrame.from_dict(price)
        price_DF['200_MA'] = price_DF[company].rolling(window=200).mean()
        price_DF['150_MA'] = price_DF[company].rolling(window=150).mean()
        price_DF['50_MA'] = price_DF[company].rolling(window=50).mean()
        price_DF['RS'] = (price_DF[company][-1]/price_DF['^GSPC'][-1])/ (price_DF[company][-252]/price_DF['^GSPC'][-252]) *100
        
        print (price_DF['200_MA'])
        
        #metrics[company]['200 MA'] = price_DF['200_MA'][–1]
        #metrics[company]['150 MA'] = price_DF['150_MA'][–1]
        #metrics[company]['50 MA'] = price_DF['50_MA'][–1]
        #metrics[company]['200 MA_1mago'] = price_DF['200_MA'][–30]
        #metrics[company]['150 MA_1mago'] = price_DF['150_MA'][–30]
        #metrics[company]['200 MA_2mago'] = price_DF['200_MA'][–60]
        #metrics[company]['150 MA_2mago'] = price_DF['150_MA'][–60]
        #metrics[company]['52W_Low'] = price_DF[company][–252:].min()
        #metrics[company]['52W_High'] = price_DF[company][–252:].max()
        #metrics[company]['price'] = price_DF[company][–1]
        #metrics[company]['Relative Strength'] = price_DF['RS'][–1]
        #Current Price is at least 30% above 52 week low (1.3*low_of_52week)
        #metrics[company]['Above_30%_low'] = metrics[company]['52W_Low'] *1.3
        # Condition 7: Current Price is within 25% of 52 week high   (.75*high_of_52week)
        #metrics[company]['Within_25%_high'] = metrics[company]['52W_High'] * 0.75
        
    except:
        pass
    
    metrics_DF = pd.DataFrame.from_dict(metrics)
    metrics_DF = metrics_DF.T 
    #to determine the rank percentil and see which are the 80% top performers
    metrics_DF['pct_rank'] = metrics_DF['Relative Strength'].rank(pct=True)
    metrics_DF = metrics_DF.T
    metrics_DF.to_csv('all_stocks_SP500.csv')
    print(metrics_DF)
    

        

count  1


KeyError: 'Relative Strength'